# Week 7 — Introduction to TensorBoard (Keras/TensorFlow)
This notebook will guide you through the 4 tasks.

In [1]:
import tensorflow as tf, os
print('TF version:', tf.__version__)
os.makedirs('runs', exist_ok=True)
os.makedirs('artifacts', exist_ok=True)

TF version: 2.20.0


## Task 1 — Basic TensorBoard Setup and Logging
We use MNIST to focus on TensorBoard features.

In [2]:

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np, os
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
VAL_RATIO = 0.1
n_val = int(len(x_train)*VAL_RATIO)
x_val, y_val = x_train[:n_val], y_train[:n_val]
x_train, y_train = x_train[n_val:], y_train[n_val:]
print("Train:", x_train.shape, y_train.shape, "| Val:", x_val.shape, y_val.shape, "| Test:", x_test.shape, y_test.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Train: (54000, 28, 28) (54000,) | Val: (6000, 28, 28) (6000,) | Test: (10000, 28, 28) (10000,)


In [3]:

def make_mlp(input_shape=(28,28), num_classes=10, hidden_units=128, dropout=0.2):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Flatten(name="flatten"),
        layers.Dense(hidden_units, activation="relu", name="dense_1"),
        layers.Dropout(dropout, name="dropout_1"),
        layers.Dense(num_classes, activation="softmax", name="logits")
    ], name=f"mlp_{hidden_units}")
    return model

def compile_model(model, learning_rate=1e-3):
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

import time
from datetime import datetime
def get_run_dir(prefix, **hparams):
    stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    tag = "__".join([f"{k}-{v}" for k,v in hparams.items()])
    dirname = f"{prefix}-{stamp}" + (f"__{tag}" if tag else "")
    run_dir = os.path.join("runs", dirname)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir


In [10]:

mlp = compile_model(make_mlp(hidden_units=128), learning_rate=1e-3)
mlp.summary()
run_dir = get_run_dir("task1_mlp", lr=1e-3, hu=128)

import os, uuid
from pathlib import Path
import tensorflow as tf
# Choose a short, ASCII-only base dir
BASE = Path("C:/tb_logs")   # or: Path(os.getenv("TEMP", "C:/Temp")) / "tb_logs"
BASE.mkdir(parents=True, exist_ok=True)

run_dir = BASE / f"task1_mlp-{uuid.uuid4().hex}"
run_dir.mkdir(parents=True, exist_ok=True)

print("Logging to:", run_dir)

tb = keras.callbacks.TensorBoard(
    log_dir=str(run_dir), histogram_freq=1, write_graph=True
)
#tb = keras.callbacks.TensorBoard(log_dir=run_dir, histogram_freq=1, write_graph=True)
history = mlp.fit(x_train, y_train, validation_data=(x_val, y_val),
                  epochs=3, batch_size=128, callbacks=[tb], verbose=2)
print("Logs ->", run_dir)
print("Files:", os.listdir(run_dir)[:10])


Model: "mlp_128"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

Logging to: C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20
Epoch 1/3
422/422 - 5s - 13ms/step - accuracy: 0.8796 - loss: 0.4215 - val_accuracy: 0.9470 - val_loss: 0.1970
Epoch 2/3
422/422 - 3s - 6ms/step - accuracy: 0.9418 - loss: 0.1993 - val_accuracy: 0.9615 - val_loss: 0.1375
Epoch 3/3
422/422 - 3s - 8ms/step - accuracy: 0.9566 - loss: 0.1492 - val_accuracy: 0.9697 - val_loss: 0.1113
Logs -> C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20
Files: ['train', 'validation']


**Open TensorBoard:** In a terminal, run: `tensorboard --logdir runs --port 6006` and open http://localhost:6006 → Scalars tab.

In [11]:
import os, glob, pprint
print("run_dir =", run_dir)

# What’s inside that folder?
for root, dirs, files in os.walk(str(run_dir)):
    print(root)
    for f in files:
        print("  -", f)

# Specifically look for TF event files
event_files = glob.glob(os.path.join(str(run_dir), "**", "events.out.tfevents.*"), recursive=True)
print("\n# of event files:", len(event_files))
pprint.pp(event_files[:5])

run_dir = C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20
C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20
C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20\train
  - events.out.tfevents.1761722940.MSI.35616.0.v2
C:\tb_logs\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20\validation
  - events.out.tfevents.1761722945.MSI.35616.1.v2

# of event files: 2
['C:\\tb_logs\\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20\\train\\events.out.tfevents.1761722940.MSI.35616.0.v2',
 'C:\\tb_logs\\task1_mlp-0488b2988ca64df29cdd5970fe8c5b20\\validation\\events.out.tfevents.1761722945.MSI.35616.1.v2']


# Task 2 — Training Metrics Visualization
We compare learning rates and log custom macro precision/recall/F1.

In [12]:

from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
import numpy as np, os

class SklearnMetrics(keras.callbacks.Callback):
    def __init__(self, val_data, log_dir):
        super().__init__()
        self.x_val, self.y_val = val_data
        self.writer = tf.summary.create_file_writer(os.path.join(log_dir, "custom_metrics"))
    def on_epoch_end(self, epoch, logs=None):
        y_prob = self.model.predict(self.x_val, verbose=0)
        y_pred = np.argmax(y_prob, axis=1)
        precision = precision_score(self.y_val, y_pred, average="macro", zero_division=0)
        recall    = recall_score(self.y_val, y_pred, average="macro", zero_division=0)
        f1        = f1_score(self.y_val, y_pred, average="macro", zero_division=0)
        with self.writer.as_default():
            tf.summary.scalar("precision_macro", precision, step=epoch)
            tf.summary.scalar("recall_macro", recall, step=epoch)
            tf.summary.scalar("f1_macro", f1, step=epoch)
        print(f"[Val epoch {epoch+1}] precision={precision:.4f} recall={recall:.4f} f1={f1:.4f}")


In [15]:
# --- Force a clean ASCII base for ALL logs ---
from pathlib import Path
import os, json, itertools
import tensorflow as tf
from tensorflow import keras

LOG_BASE = Path("C:/tb_logs").resolve()
LOG_BASE.mkdir(parents=True, exist_ok=True)
print("LOG_BASE =", LOG_BASE)

def get_run_dir(prefix, **hparams):
    from datetime import datetime
    stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    tag = "__".join([f"{k}-{v}" for k,v in hparams.items()])
    name = f"{prefix}-{stamp}" + (f"__{tag}" if tag else "")
    run_dir = LOG_BASE / name
    # ensure directory exists and is a dir
    if run_dir.exists() and not run_dir.is_dir():
        run_dir.unlink()
    run_dir.mkdir(parents=True, exist_ok=True)
    return run_dir  # return a Path

# Re-define the callback to create its subdir safely, using POSIX paths
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

class SklearnMetrics(keras.callbacks.Callback):
    def __init__(self, val_data, log_dir):
        super().__init__()
        self.x_val, self.y_val = val_data
        cm_dir = Path(log_dir) / "custom_metrics"
        if cm_dir.exists() and not cm_dir.is_dir():
            cm_dir.unlink()
        # make sure it exists and pass a POSIX string to TF
        tf.io.gfile.makedirs(cm_dir.as_posix())
        self.writer = tf.summary.create_file_writer(cm_dir.as_posix())

    def on_epoch_end(self, epoch, logs=None):
        y_prob = self.model.predict(self.x_val, verbose=0)
        y_pred = np.argmax(y_prob, axis=1)
        precision = precision_score(self.y_val, y_pred, average="macro", zero_division=0)
        recall    = recall_score(self.y_val, y_pred, average="macro", zero_division=0)
        f1        = f1_score(self.y_val, y_pred, average="macro", zero_division=0)
        with self.writer.as_default():
            tf.summary.scalar("precision_macro", precision, step=epoch)
            tf.summary.scalar("recall_macro",    recall,    step=epoch)
            tf.summary.scalar("f1_macro",        f1,        step=epoch)

def train_one_run(lr=1e-3, hidden_units=128, batch_size=128, epochs=5, prefix="task2_mlp"):
    model = compile_model(make_mlp(hidden_units=hidden_units), learning_rate=lr)

    run_dir = get_run_dir(prefix, lr=lr, hu=hidden_units, bs=batch_size)
    # Ensure the Keras TB callback also gets a POSIX (ASCII) path:
    tb = keras.callbacks.TensorBoard(log_dir=run_dir.as_posix(), histogram_freq=1, write_graph=True)

    callbacks = [
        tb,
        SklearnMetrics(val_data=(x_val, y_val), log_dir=run_dir),
        keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        keras.callbacks.ModelCheckpoint(
            (Path("artifacts") / f"{prefix}_lr{lr}_hu{hidden_units}_bs{batch_size}.keras").as_posix(),
            monitor="val_accuracy", save_best_only=True
        ),
        keras.callbacks.CSVLogger((run_dir / "history.csv").as_posix())
    ]

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs, batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )
    print("Logs ->", run_dir)
    return run_dir, history.history


LOG_BASE = C:\tb_logs


In [16]:

run_a, hist_a = train_one_run(lr=1e-2, hidden_units=128, batch_size=128, epochs=6, prefix="task2_mlp")
run_b, hist_b = train_one_run(lr=1e-3, hidden_units=128, batch_size=128, epochs=6, prefix="task2_mlp")
print("Compare runs in TensorBoard:", run_a, "vs", run_b)


Epoch 1/6
422/422 - 5s - 12ms/step - accuracy: 0.9173 - loss: 0.2711 - val_accuracy: 0.9607 - val_loss: 0.1305
Epoch 2/6
422/422 - 4s - 9ms/step - accuracy: 0.9485 - loss: 0.1687 - val_accuracy: 0.9678 - val_loss: 0.1123
Epoch 3/6
422/422 - 4s - 8ms/step - accuracy: 0.9571 - loss: 0.1417 - val_accuracy: 0.9682 - val_loss: 0.1057
Epoch 4/6
422/422 - 3s - 8ms/step - accuracy: 0.9614 - loss: 0.1229 - val_accuracy: 0.9678 - val_loss: 0.1201
Epoch 5/6
422/422 - 4s - 8ms/step - accuracy: 0.9647 - loss: 0.1162 - val_accuracy: 0.9657 - val_loss: 0.1394
Epoch 6/6
422/422 - 3s - 8ms/step - accuracy: 0.9657 - loss: 0.1122 - val_accuracy: 0.9702 - val_loss: 0.1246
Logs -> C:\tb_logs\task2_mlp-20251029-145733__lr-0.01__hu-128__bs-128
Epoch 1/6
422/422 - 5s - 11ms/step - accuracy: 0.8806 - loss: 0.4249 - val_accuracy: 0.9462 - val_loss: 0.1977
Epoch 2/6
422/422 - 4s - 9ms/step - accuracy: 0.9401 - loss: 0.2055 - val_accuracy: 0.9592 - val_loss: 0.1425
Epoch 3/6
422/422 - 4s - 9ms/step - accuracy: 0.

**Write your insights here:** Which LR converged faster? Do custom metrics differ from accuracy? Any overfitting/underfitting signs?

# Task 2 — Training Progress & Metric Insights

## Scalars (training)
- **Accuracy (`epoch_accuracy`)** rises steadily from ~**0.91–0.92** at epoch 0 to ~**0.96** by epoch 5.
  - Shape shows **fast early gains** (epochs 0→2) and **diminishing returns** after epoch 3.
- **Loss (`epoch_loss`)** drops from ~**0.24–0.29** at epoch 0 to ~**0.12** by epoch 5.
  - The curve is smooth and monotonic ↓, indicating **stable optimization** (no spikes/instability).

**Takeaway:** The learning rate is well-tuned for this model/data; training progresses smoothly with no obvious divergence.

---

## Custom Validation Metrics (macro)
*(logged via the `SklearnMetrics` callback)*

- **F1 (macro):** climbs from ~**0.94–0.96** at epoch 0 to ~**0.969–0.971** at epoch 5.
- **Precision (macro):** increases from ~**0.95–0.96** to ~**0.969–0.970** by epoch 5.
- **Recall (macro):** improves from ~**0.94–0.95** to ~**0.970–0.972** by epoch 5.

**Class balance insight:** Precision, recall, and F1 remain **close to each other** throughout training, suggesting **no major class imbalance issues** and that the model is **not favoring specific digits**. Slightly faster growth in precision/recall early on indicates the decision boundaries improve quickly, then refine more slowly.

---

# Task 3 — Model Architecture Visualization (Graphs tab)
We log graphs for a deeper MLP and a simple CNN and compare them.

In [18]:

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf, os

def make_deeper_mlp(input_shape=(28,28), num_classes=10):
    return keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax")
    ], name="deeper_mlp")

def make_cnn(input_shape=(28,28,1), num_classes=10):
    return keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Reshape((28,28,1)),
        layers.Conv2D(32, 3, activation="relu", padding="same"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu", padding="same"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax")
    ], name="simple_cnn")

# --- Patch for Task 3: make sure TF gets a string logdir ---
from pathlib import Path
import tensorflow as tf
from tensorflow import keras

# Ensure a clean ASCII base on Windows
LOG_BASE = Path("C:/tb_logs").resolve()
LOG_BASE.mkdir(parents=True, exist_ok=True)

def get_run_dir(prefix):
    from datetime import datetime
    name = f"{prefix}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    rd = (LOG_BASE / name)
    rd.mkdir(parents=True, exist_ok=True)
    return rd  # Path

def train_with_graph(model, run_prefix, epochs=3):
    model = compile_model(model, learning_rate=1e-3)
    run_dir = get_run_dir(run_prefix)               # Path
    logdir  = run_dir.as_posix()                    # <-- convert to string
    tf.io.gfile.makedirs(logdir)

    tb = keras.callbacks.TensorBoard(
        log_dir=logdir, histogram_freq=1, write_graph=True
    )

    # Force graph trace to appear in the Graphs tab
    tf.summary.trace_on(graph=True, profiler=False)

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs, batch_size=128,
        callbacks=[tb], verbose=2
    )

    # IMPORTANT: pass string path to the writer
    with tf.summary.create_file_writer(logdir).as_default():
        tf.summary.trace_export(name="graph_trace", step=0)

    print("Graphs/logs ->", logdir)
    return run_dir


run_deeper = train_with_graph(make_deeper_mlp(), "task3_deeper_mlp", epochs=3)
run_cnn    = train_with_graph(make_cnn(), "task3_cnn", epochs=3)
print("Graphs written under:", run_deeper, "and", run_cnn)

# Optional: export .png diagrams if graphviz+pydot are available
try:
    keras.utils.plot_model(make_deeper_mlp(), to_file=os.path.join("artifacts", "deeper_mlp.png"), show_shapes=True)
    keras.utils.plot_model(make_cnn(), to_file=os.path.join("artifacts", "simple_cnn.png"), show_shapes=True)
    print("Saved diagrams to artifacts/")
except Exception as e:
    print("plot_model skipped:", e)

Epoch 1/3
422/422 - 5s - 12ms/step - accuracy: 0.9031 - loss: 0.3373 - val_accuracy: 0.9623 - val_loss: 0.1332
Epoch 2/3
422/422 - 4s - 9ms/step - accuracy: 0.9611 - loss: 0.1327 - val_accuracy: 0.9725 - val_loss: 0.0938
Epoch 3/3
422/422 - 3s - 8ms/step - accuracy: 0.9720 - loss: 0.0932 - val_accuracy: 0.9780 - val_loss: 0.0739
Graphs/logs -> C:/tb_logs/task3_deeper_mlp-20251029-150617
Epoch 1/3
422/422 - 12s - 28ms/step - accuracy: 0.9260 - loss: 0.2429 - val_accuracy: 0.9828 - val_loss: 0.0577
Epoch 2/3
422/422 - 10s - 23ms/step - accuracy: 0.9794 - loss: 0.0666 - val_accuracy: 0.9850 - val_loss: 0.0482
Epoch 3/3
422/422 - 20s - 47ms/step - accuracy: 0.9857 - loss: 0.0469 - val_accuracy: 0.9887 - val_loss: 0.0414
Graphs/logs -> C:/tb_logs/task3_cnn-20251029-150629
Graphs written under: C:\tb_logs\task3_deeper_mlp-20251029-150617 and C:\tb_logs\task3_cnn-20251029-150629
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
You 

**Explain the structures:** Deeper MLP (dense-only) vs CNN (convs + pooling). In Graphs tab, inspect the tensor flow differences.

# Task 3 — Model Architecture & Data Flow

## 1) Deeper MLP (Dense-only)

**Purpose:** Fast baseline for 2D images when you’re OK with flattening (loses spatial structure).

**Layers & data flow**
1. **Input**: `(28, 28)` grayscale image  
2. **Flatten** → `(784,)` — collapses H×W into a 1D vector (destroys locality)
3. **Dense(256, ReLU)** — learns global combinations of pixels
4. **Dense(128, ReLU)** — refines higher-level combinations
5. **Dropout(0.3)** — regularizes (randomly zeroes activations during training)
6. **Dense(10, Softmax)** — class probabilities for digits 0–9

**Key characteristics**
- **Pros:** Simple, fast to train, good for demos/ablation and tabular-like inputs.  
- **Cons:** Ignores spatial structure; must rely on many parameters to “rediscover” locality.  
- **When to use:** Quick baselines, sanity checks, or when input is already 1D features.

---

## 2) Simple CNN (Convolutional Neural Network)

**Purpose:** Exploit image **spatial locality** and **translation invariance** for better vision performance.

**Layers & data flow**
1. **Input**: `(28, 28)` → **Reshape** to `(28, 28, 1)` (add channel dim)
2. **Conv2D(32, 3×3, ReLU, same)** → `(28, 28, 32)`  
   - 32 learned 3×3 filters slide over the image to detect local patterns (edges, corners)
3. **MaxPool(2×2)** → `(14, 14, 32)`  
   - Downsamples; retains strongest activations → translation tolerance
4. **Conv2D(64, 3×3, ReLU, same)** → `(14, 14, 64)`  
   - Deeper filters capture more complex patterns (strokes, parts of digits)
5. **MaxPool(2×2)** → `(7, 7, 64)`
6. **Flatten** → `(3136,)`
7. **Dense(128, ReLU)** — combines learned local features into global representation
8. **Dropout(0.3)** — regularization
9. **Dense(10, Softmax)** — class probabilities

**Key characteristics**
- **Pros:** Uses convolutions to capture **local structure** efficiently; usually higher accuracy on vision tasks with fewer parameters than a similarly strong MLP.  
- **Cons:** Slightly more complex; may need more tuning (filters, pooling strategy).  
- **When to use:** Any image data; preferred default for vision.

---

## 3) Side-by-Side Comparison

| Aspect | Deeper MLP | Simple CNN |
|---|---|---|
| **Input handling** | Flattens `(28,28)` → `(784,)` | Keeps spatial grid `(28,28,1)` |
| **Inductive bias** | None for images; treats all pixels as independent positions | Strong locality & translation bias via convolution/pooling |
| **Representation** | Global dense features | Hierarchical local → global features |
| **Parameter efficiency** | Typically **more** params for similar accuracy | Often **fewer** params with **better** accuracy |
| **Performance on MNIST** | Solid baseline (~96–98% with tuning) | Typically better (often 98–99%+) |
| **Overfitting risk** | Higher if widened deeply without regularization | Lower for same capacity due to weight sharing |
| **Train speed** | Very fast per step | Slightly slower per step; converges in fewer epochs |
| **Best use case** | Quick baselines, 1D/tabular features | Image data (default choice) |

**Why CNN wins on images**
- Convolutions **share weights** across locations → learn edge/curve detectors once and reuse them everywhere.  
- Pooling provides **translation tolerance** and reduces spatial resolution, focusing on salient features.  
- MLP must learn similar patterns **independently** across positions after flattening, which is less efficient and less robust.

---


# Task 4 — Hyperparameter Tuning and Comparison
Grid search over LR, batch size, and hidden units; summarize best run.

In [20]:
# Unify get_run_dir so Task 2 & 4 work, and keep logs in C:/tb_logs
from pathlib import Path
from datetime import datetime
import tensorflow as tf

LOG_BASE = Path("C:/tb_logs").resolve()
LOG_BASE.mkdir(parents=True, exist_ok=True)
print("LOG_BASE =", LOG_BASE)

def get_run_dir(prefix, **hparams):
    stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    tag = "__".join([f"{k}-{v}" for k, v in hparams.items()]) if hparams else ""
    name = f"{prefix}-{stamp}" + (f"__{tag}" if tag else "")
    rd = LOG_BASE / name
    # ensure it's a directory
    if rd.exists() and not rd.is_dir():
        rd.unlink()
    rd.mkdir(parents=True, exist_ok=True)
    return rd  # return Path

# (Optional) small helpers to always pass strings to TF
def as_logdir(p: Path) -> str:
    return p.as_posix()


LOG_BASE = C:\tb_logs


In [21]:

import itertools, os, pandas as pd, json, shutil

def run_grid_search(lrs, bss, hus, epochs=6):
    results = []
    best = {"val_accuracy": -1.0, "run_dir": None, "hparams": None}
    for lr, bs, hu in itertools.product(lrs, bss, hus):
        run_dir, _ = train_one_run(lr=lr, hidden_units=hu, batch_size=bs, epochs=epochs, prefix="task4_grid_mlp")
        csv_path = os.path.join(run_dir, "history.csv")
        hist = pd.read_csv(csv_path)
        best_val_acc = hist["val_accuracy"].max()
        results.append({"lr": lr, "batch_size": bs, "hidden_units": hu, "best_val_accuracy": best_val_acc, "run_dir": run_dir})
        if best_val_acc > best["val_accuracy"]:
            best = {"val_accuracy": float(best_val_acc), "run_dir": run_dir, "hparams": {"lr": lr, "batch_size": bs, "hidden_units": hu}}
    df = pd.DataFrame(results).sort_values(by="best_val_accuracy", ascending=False).reset_index(drop=True)
    return df, best

grid_df, best = run_grid_search(lrs=[1e-3, 5e-4], bss=[64, 128], hus=[128, 256], epochs=6)
grid_df


Epoch 1/6
844/844 - 7s - 9ms/step - accuracy: 0.8982 - loss: 0.3529 - val_accuracy: 0.9552 - val_loss: 0.1637
Epoch 2/6
844/844 - 6s - 7ms/step - accuracy: 0.9505 - loss: 0.1702 - val_accuracy: 0.9675 - val_loss: 0.1187
Epoch 3/6
844/844 - 6s - 7ms/step - accuracy: 0.9624 - loss: 0.1267 - val_accuracy: 0.9723 - val_loss: 0.0955
Epoch 4/6
844/844 - 5s - 6ms/step - accuracy: 0.9696 - loss: 0.1018 - val_accuracy: 0.9762 - val_loss: 0.0835
Epoch 5/6
844/844 - 5s - 6ms/step - accuracy: 0.9726 - loss: 0.0866 - val_accuracy: 0.9762 - val_loss: 0.0779
Epoch 6/6
844/844 - 5s - 6ms/step - accuracy: 0.9771 - loss: 0.0746 - val_accuracy: 0.9777 - val_loss: 0.0741
Logs -> C:\tb_logs\task4_grid_mlp-20251029-152303__lr-0.001__hu-128__bs-64
Epoch 1/6
844/844 - 7s - 9ms/step - accuracy: 0.9117 - loss: 0.3045 - val_accuracy: 0.9628 - val_loss: 0.1388
Epoch 2/6
844/844 - 7s - 8ms/step - accuracy: 0.9600 - loss: 0.1365 - val_accuracy: 0.9718 - val_loss: 0.0964
Epoch 3/6
844/844 - 9s - 11ms/step - accuracy

,lr,batch_size,hidden_units,best_val_accuracy,run_dir
0,0.0010,64,256,0.980667,C:\tb_logs\task4_grid_mlp-20251029-152337__lr-...
1,0.0010,128,256,0.977833,C:\tb_logs\task4_grid_mlp-20251029-152453__lr-...
2,0.0010,64,128,0.977667,C:\tb_logs\task4_grid_mlp-20251029-152303__lr-...
3,0.0005,64,256,0.975833,C:\tb_logs\task4_grid_mlp-20251029-152621__lr-...
4,0.0010,128,128,0.975500,C:\tb_logs\task4_grid_mlp-20251029-152428__lr-...
5,0.0005,128,256,0.975000,C:\tb_logs\task4_grid_mlp-20251029-152724__lr-...
6,0.0005,64,128,0.973333,C:\tb_logs\task4_grid_mlp-20251029-152534__lr-...
7,0.0005,128,128,0.969000,C:\tb_logs\task4_grid_mlp-20251029-152659__lr-...


**Best configuration (this grid):**  
- `learning_rate = 1e-3`  
- `batch_size = 64`  
- `hidden_units = 256`  
**Result:** Validation accuracy ≈ **0.9807**

**Grid ranges tested:**  
- `lr ∈ {1e-3, 5e-4}`  
- `batch_size ∈ {64, 128}`  
- `hidden_units ∈ {128, 256}`

---

## What the grid shows

### Learning Rate (LR)
- **1e-3 > 5e-4** across almost all settings (top 3 runs all use `1e-3`).
- **Reasoning:** MNIST is well-conditioned; a slightly larger LR speeds convergence without overshooting. With `5e-4`, curves rise slower and plateau slightly lower.

### Batch Size
- **64 ≥ 128** by a small, consistent margin at the same LR/width.
- **Reasoning:** Smaller batches add gradient noise that can improve generalization (flatter minima). `128` is smoother but a touch lower in final val accuracy.

### Hidden Units (Model Width)
- **256 > 128** at both LRs and both batch sizes.
- **Reasoning:** Wider MLP better captures digit-stroke variation after flattening. No overfitting observed under current epochs/regularization.

### Interaction Effects
- **LR × Width:** Gains from 256 units are clearest when **LR = 1e-3** (capacity is actually utilized).
- **LR × Batch:** With **1e-3**, batch **64** typically edges **128**; with **5e-4**, differences narrow because the step size limits progress.

---

In [ ]:

# Save artifacts
summary_path = os.path.join("artifacts", "task4_grid_summary.csv")
grid_df.to_csv(summary_path, index=False)
with open(os.path.join("artifacts", "task4_best.json"), "w") as f:
    json.dump(best, f, indent=2)
print("Summary saved to:", summary_path)
print("Best config:", best)

# Copy best logs to a convenient folder
BEST_LOGS_DIR = os.path.join("runs", "best_run_logs")
if os.path.exists(BEST_LOGS_DIR):
    shutil.rmtree(BEST_LOGS_DIR)
shutil.copytree(best["run_dir"], BEST_LOGS_DIR)
print("Best run logs copied to:", BEST_LOGS_DIR)


## Evaluation Criteria Mapping
- TensorBoard Setup: descriptive run names + logs in `runs/`.
- Training Metrics: Scalars + custom macro precision/recall/F1.
- Model Architecture: Graphs tab (MLP vs CNN) + optional diagrams.
- Hyperparameter Tuning: grid, CSV logs, best config JSON.
- Code Quality: functions, seeds, checkpoints, early stopping.

## Summary & Recommendations (fill after running)
- Paste key screenshots and write 3–5 bullet recommendations here.